In [1]:
# Check that we have a GPU
!nvidia-smi

Fri Jul 10 20:06:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   34C    P8    10W / 275W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0 Off |                  N/A |
|  0%   33C    P8    10W / 275W |     18MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [3]:
# make output folder
!mkdir DistilBert

In [4]:
train_path='../wikitext-2-raw/wiki.train.raw'
TEST_FILE='../wikitext-2-raw/wiki.test.raw'
eval_path= '../wikitext-2-raw/wiki.valid.raw'

In [5]:

import os
#Setting environment variables
os.environ["train_path"] = train_path
os.environ["eval_path"] = eval_path
os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)
weights_dir = "DistilBert/weights"
!mkdir {weights_dir}

In [12]:
cmd = '''python ../transformers/examples/language-modeling/run_language_modeling.py --output_dir {0}  \
    --model_type distilbert \
    --model_name_or_path distilbert-base-uncased \
    --mlm \
    --train_data_file {1} \
    --eval_data_file {2} \
    --do_train \
    --line_by_line \
    --overwrite_output_dir \
    --do_eval \
    --block_size 128 \
    --learning_rate 2e-4 \
    --num_train_epochs 1 \
    --save_total_limit 2 \
    --save_steps 20000 \
    --logging_steps 1000 \
    --per_device_eval_batch_size 16 \
    --per_device_train_batch_size 16 \
    --evaluate_during_training \
    --seed 42 \
    '''.format(weights_dir, train_path, eval_path)

In [ ]:
!{cmd}

## using Trainer

In [14]:
TRAIN_FILE='../wikitext-2-raw/wiki.train.raw'
TEST_FILE='../wikitext-2-raw/wiki.test.raw'

In [16]:
from transformers import DistilBertForMaskedLM
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased') #this works
model.num_parameters()

66985530

In [17]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [18]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_FILE,
    block_size=12,
)

CPU times: user 20.9 s, sys: 0 ns, total: 20.9 s
Wall time: 20.9 s


In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=weights_dir,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
%%time
trainer.train()